# Diseño Básico de Vigas de Hormigón Armado en Python
## Proyecto: Aplicación para diseño estructural (Flexión, Cortante y Flexocompresión)
### 1. INTRODUCCIÓN:
El objetivo de este trabajo es desarrollar una aplicación en Python (Jupyter Notebook) que realice el diseño básico de vigas de hormigón armado sometidas a esfuerzos de flexión, cortante y flexo-compresión, aplicando los criterios de resistencia admisible de las normas ACI 318 / CIRSOC 201.

Además, se busca incorporar un módulo de optimización que seleccione la sección más eficiente en términos estructurales y económicos.




### 2. OBJETIVOS:
- Calcular las solicitaciones (momentos y cortantes) aplicadas a una viga.

- Determinar la capacidad resistente del elemento a flexión y cortante.

- Calcular la cantidad de acero de refuerzo necesaria para cumplir con la resistencia requerida.

- Verificar los estados límite de servicio.

- Proponer una sección óptima según resistencia y costo.

- Exportar los resultados en Excel o PDF.

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt

### 3. FUNDAMENTOS TEÓRICOS:
#### 3.1. Diagrama de esfuerzos: 
3.1.1. Para una viga simplemente apoyada con carga distribuida $w$ y carga puntual $P$, el Momento y el cortante, en una posición dada de la barra (de longitud $l$), es:
$$M(x) = R_A*x - \frac{w*x^2}{2}$$
$$V(x) = R_A - {w*x}$$
$$R_A = R_B$$
- $R_A$ y $R_B$ son las reacciones en los apoyos/extremos de la viga.
- $V(x)$: Fuerza Cortante en la sección "$x$".
- $M(x)$: Momento Cortante en la sección "$x$".

El momento máximo ocurre en el centro de la luz: 
$$M_{máx} = \frac{w*L^2}{8}$$

El cortante máximo ocurre en el apoyo: 
$$V_{máx} = R_A = R_B$$


In [15]:
# Fuerzas en N: Newtons
# Longitud en mm: milímetros
# Momentos en N.mm: Newtons por milímetros
# Tensiones y/o esfuerzos en N.mm^2: Newtons por milímetros cuadrados
def fuerzas(L, P = None, w = 0.0): # L = longitud de la barra. P = Carga Puntual. w = carga distribuída. Inicialmente P y w son ambos de valor cero.
    
    # P es una lista de tuplas (x, P_x): Posición y valor de la carga puntual

    # Reacciones en los apoyos por carga distribuída:
    RA = w * L / 2
    RB = RA
    
    # Reacciones en los apoyos por cargas puntuales:
    if P:
        for x, P_x in P:
            RA += P_x * (L - x) / L
            RB += P_x * x / L
    xs = np.linspace(0, L, 501)
    V = np.zeros_like(xs)
    M = np.zeros_like(xs)
    for i, x in enumerate(xs):
        v = 0.0 # Iniciar las reacciones
        m = 0.0 # Iniciar el momento
        v += RA
        m += RA * x
    
    # Restar la carga distribuída:
    v -= w * x
    m -= w * x * x / 2
    
    # Restar las cargas puntuales a la derecha de x:
    if P:
        for xp, P_x in P:
                if xp <= x:
                    v -= P_x
                    m -= P_x * (x - xp)
    
    # Agregar reacción RB si x > location of RB (RB actúa en soporte B en x = L)
    if x >= L:
        v += RB
        m += RB * (x - L)
        V[i] = v
        M[i] = m
    return xs, V, M, (RA, RB)


In [8]:
#x = np.linspace(0, 10, 100)
#y = np.sin(x)
#plt.plot(x,y)
#plt.title('Ejemplo simple')
#plt.show()